In [1]:
# Libraries Loading
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from matplotlib.ticker import MaxNLocator
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (average_precision_score, confusion_matrix,
                             f1_score, log_loss, precision_score, recall_score,
                             roc_auc_score)
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import roc_auc_score

# read in data from PCA
X_train_pca = pd.read_pickle('PCA_X_train.pkl')
y_train_pca = pd.read_pickle('PCA_y_train.pkl')
X_test_pca = pd.read_pickle('PCA_X_test.pkl')
y_test_pca = pd.read_pickle('PCA_y_test.pkl')


# read in data from Autoencoder
X_train_autoencoder = pd.read_pickle('PCA_X_train.pkl')
y_train_autoencoder = pd.read_pickle('PCA_y_train.pkl')
X_test_autoencoder = pd.read_pickle('PCA_X_test.pkl')
y_test_autoencoder = pd.read_pickle('PCA_y_test.pkl')

In [2]:
url = 'https://drive.google.com/file/d/1E9bFoMAwMVPMkZHmMQfWvSgaWoCLtKp4/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_test_autoencoder = pd.read_pickle(url)

url = 'https://drive.google.com/file/d/1dknbQLJhQsbSFDNIpwjxZQwPc3fF14yF/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_train_autoencoder = pd.read_pickle(url)


url = 'https://drive.google.com/file/d/105vmXogIQU8srmtdzfCHYQ9G-T7Iu5hl/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
X_test_autoencoder = pd.read_pickle(url)

url = 'https://drive.google.com/file/d/1QemU274E0c14R8uknuipP20rNgDSLzcg/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
X_train_autoencoder = pd.read_pickle(url)

print('1')


1


In [4]:
y_test_autoencoder.shape

(103495,)

In [5]:
y_train_autoencoder.shape

(796896,)

In [6]:
X_test_autoencoder.shape

(103495, 10)

In [8]:
X_train_autoencoder.shape

(796896, 10)

## Random Forest (Autoencoder)

In [10]:
# Create random forest object 
rf = RandomForestClassifier(
    criterion='gini',
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=190322)

# Define hyperparameters 
n_estimators = [50, 100]
min_samples_leaf = [2,4,5]
#max_depth = [2,4,5]
# max_features = [1,2,3,4,5]


# dictionary of hyperparams 
hyperparms = dict(n_estimators = n_estimators, 
                 min_samples_leaf = min_samples_leaf, 
                 max_depth = max_depth)


# Use gridsearch to fit the model 
cv_hyper = GridSearchCV(rf, hyperparms, cv = 3, verbose = 0)
best_mod = cv_hyper.fit(X_train_autoencoder, y_train_autoencoder)        


# Print best hyperparams 
print('Best hyperparameters:', best_mod.best_params_)
print('Best cross-validation AUC score:', best_mod.best_score_)

Best hyperparameters: {'max_depth': 5, 'min_samples_leaf': 5, 'n_estimators': 100}
Best cross-validation AUC score: 0.958807673774244


In [11]:
# create random forest object with best hyperparams
rf_model = RandomForestClassifier(
    n_estimators = best_mod.best_estimator_.get_params()['n_estimators'], 
    min_samples_leaf = best_mod.best_estimator_.get_params()['min_samples_leaf'], 
    max_depth = best_mod.best_estimator_.get_params()['max_depth'], 
    criterion='gini',
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=190322)

# Estimate model 
rf_model.fit(X_train_autoencoder, y_train_autoencoder)

# Predictions for test set 
yhat_test = rf_model.predict(X_test_autoencoder)
# AUC
ROC_AUC_randomFroest = roc_auc_score(y_test_autoencoder, yhat_test)

print('ROC AUC for Random Forest in test dataset : {:.4f}'.format(ROC_AUC_randomFroest))

ROC AUC for Random Forest in test dataset : 0.8755


In [16]:
rf_model

RandomForestClassifier(max_depth=5, min_samples_leaf=5, n_jobs=-1,
                       oob_score=True, random_state=190322)

In [13]:
import pickle

In [14]:
#convert to pickle file for latter running
#pickle.dump(rf_model, open('RandomForestModel.pkl', 'wb'))

In [18]:
# Load the model from the pickle file
with open('RandomForestModel.pkl', 'rb') as f:
    model = pickle.load(f)

In [19]:
model

RandomForestClassifier(max_depth=5, min_samples_leaf=5, n_jobs=-1,
                       oob_score=True, random_state=190322)

In [20]:
#testing chunk 

yhat_test_pickle = model.predict(X_test_autoencoder)

ROC_AUC_randomFroest = roc_auc_score(y_test_autoencoder, yhat_test_pickle)

print('ROC AUC for Random Forest in test dataset : {:.4f}'.format(ROC_AUC_randomFroest))

ROC AUC for Random Forest in test dataset : 0.8755
